# Integrating in Python
Ok, so you need to do an integral. You are using Python, so you decide to use quad() from scipy.integrate. The thing is, your integral ends up taking forever. How can we speed this up? Well, the answer is to write your function in C and call it from in Python. Let's see how.

In [1]:
#Import everything we need
import numpy as np
import time
from scipy.integrate import quad

Our integrand is $\cos(x)/x^3$ evaluated from $\pi$ to $2\pi$. Yes, I know, this can be done analytically. This is just for demonstrative purposes. Pretend it was more complicated. The answer to this integral is approximately -0.0152115.

In [2]:
def integrand(x):
    return np.cos(x)/x**3

We now have our integrand in Python-function form. Let's time how long it takes to do this integral one hundred thousand times.

In [3]:
start = time.time()
for i in range(100000):
    answer = quad(integrand, np.pi, 2*np.pi)
end = time.time()
print "Got answer: ", answer
print "Time with Python integrand: ", end - start

Got answer:  (-0.015211452880075338, 2.903565890448967e-16)
Time with Python integrand:  3.53338909149


I also have the integrand written in C in the file integrand.c. The text of the file is:

#include<math.h>

double integrand(int i, double x){
  return cos(x)/(x*x*x);
}  

To compile the file, we can use the makefile by simply calling $make. Inside the makefile we have the following:

integrand:
	gcc -shared -o integrand.so -fPIC integrand.c
    
The two flags that must be turned on to make a *shared object (.so)* file are -shared and -fPIC. Now we can get this .so object and its functions into our Python program.

In [4]:
import ctypes
lib = ctypes.cdll.LoadLibrary("integrand.so")
c_integrand = lib.integrand
c_integrand.argtypes = (ctypes.c_int, ctypes.c_double)
c_integrand.restype  = ctypes.c_double

In [5]:
start = time.time()
for i in range(100000):
    answer = quad(c_integrand, np.pi, 2*np.pi)
end = time.time()
print "Got answer: ", answer
print "Time with ctypes integrand: ", end - start

Got answer:  (-0.015211452880075338, 2.903565890448967e-16)
Time with ctypes integrand:  1.45353293419


**NOTE**: an integrand being used for quad needs the signature as we have written it: an integer followed by a double, with a double as the return type. This is for compatability with nquad(), which I'll get to in another notebook.